In [1]:
import pandas as pd
from pathlib import Path
import regex as re

In [2]:
root_folderpath = Path(r"C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau")
matrices_individual_folderpath = Path(root_folderpath / r"data\matrices_individual")
matrices_dividends_folderpath =  Path(root_folderpath / r"data\matrices_dividends")
vectors_individual_folderpath = Path(root_folderpath / r"data\vectors_individual")
vectors_consumer_flows_folderpath = Path(root_folderpath / r"data\vectors_consumer_flows")
vectors_producer_flows_folderpath = Path(root_folderpath / r"data\vectors_producer_flows")
muni_names_filepath = Path(root_folderpath / r"data\crosswalk\dk_municipalities_code_to_name.xlsx")
output_folderpath = Path(root_folderpath / r"output_csv")

In [3]:
muni_names_crosswalk = pd.read_excel(muni_names_filepath)
muni_names_crosswalk = muni_names_crosswalk.convert_dtypes()
muni_names_crosswalk.dtypes

code    string
name    string
dtype: object

# Process matrices

## Process individual matrices (that don't require special processing)

In [4]:
def convert_munis_to_names(df, muni_names_crosswalk):
    df = df.copy()
    print("Initial state:")
    print(df.shape, df.columns)
        
    # Remove columns like 'prod_cell' and 'cust_cell'
    for col in df.columns:
        if 'cell' in col:
            df = df.drop(col, axis=1)
    print("After removing 'cell' variables:")
    print(df.shape, df.columns)
        
    # Convert 'muni' columns into string
    for col in df.columns:
        if 'muni' in col:
            df[col] = df[col].astype('string')
    print(df.dtypes)

    # Convert values in sectors to integers, i.e. remove the 's' in 's01'
    for col in df.columns:
        if 'sector' in col:
            df[col] = df[col].str.replace('s','')

    # Convert munis into muni names, and drop the original munis.
    df_merge = df.copy()
    for col in df_merge.columns:
        if '_muni' in col:
            muni_name = col + '_name'
            print("Processing:", col)
            print("Before merge:", df.shape, df_merge.columns)
            df_merge = df_merge.merge(muni_names_crosswalk, left_on=col, right_on='code', how='inner', validate='m:1')
            assert(df_merge.shape[0] == df.shape[0])
            print("After merge:", df.shape, df_merge.columns)
            df_merge = df_merge.rename(columns={'name':muni_name})
            df_merge = df_merge.drop([col, 'code'], axis=1)
            print("After processing:", df.shape, df_merge.columns)
    
    return df_merge

In [5]:
def save_df_to_csv(df, input_filepath, output_folderpath, suffix=''):
    output_filename = input_filepath.stem + suffix + '.csv'
    output_filepath = Path(output_folderpath / output_filename)
    df.to_csv(output_filepath, index=False)
    print("Saved as:", output_filepath)

In [6]:
for matrix_filepath in matrices_individual_folderpath.iterdir():
    print(matrix_filepath)
    df = pd.read_csv(matrix_filepath, low_memory=False)
    
    # Create a new df where munis are converted to names, then save as csv
    df_merge = convert_munis_to_names(df, muni_names_crosswalk)
    save_df_to_csv(df_merge, matrix_filepath, output_folderpath)
    
    print('\n-')

C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\matrices_individual\Domestic consumer spending matrix (predicted).csv
Initial state:
(4571504, 7) Index(['cust_muni', 'cust_sector', 'cust_cell', 'spend_muni', 'spend_sector',
       'spend_cell', 's_'],
      dtype='object')
After removing 'cell' variables:
(4571504, 5) Index(['cust_muni', 'cust_sector', 'spend_muni', 'spend_sector', 's_'], dtype='object')
cust_muni        string
cust_sector      object
spend_muni       string
spend_sector     object
s_              float64
dtype: object
Processing: cust_muni
Before merge: (4571504, 5) Index(['cust_muni', 'cust_sector', 'spend_muni', 'spend_sector', 's_'], dtype='object')
After merge: (4571504, 5) Index(['cust_muni', 'cust_sector', 'spend_muni', 'spend_sector', 's_', 'code',
       'name'],
      dtype='object')
After processing: (4571504, 5) Index(['cust_sector', 'spend_muni', 'spend_sector', 's_', 'cust_muni_name'],

## Process the dividends matrix (that requires special processing)
Note: we will be breaking the dividends matrix up into 2 vectors. Also, even though this is written as a loop for generalization, only 1 file is expected.

In [7]:
for matrix_filepath in matrices_dividends_folderpath.iterdir():
    print(matrix_filepath)
    df = pd.read_csv(matrix_filepath, low_memory=False)
        
    # Create 2 new 'vectors', one where the producer cells are aggregated and the other where the consumer cells are aggregated
    df_producer = df.groupby(['prod_muni', 'prod_sector', 'prod_cell']).sum().reset_index().drop('cust_muni', axis=1)
    df_consumer = df.groupby(['cust_muni', 'cust_sector', 'cust_cell']).sum().reset_index().drop('prod_muni', axis=1)
    # Save the producer vector to vectors_producer_flows_folderpath and the consumer vector to vectors_consumer_flows_folderpath
    # These 2 vectors will be processed together with the existing vectors, to form the additional producer/consumer flows files. 
    save_df_to_csv(df_producer, Path('Producer dividends paid.csv'), vectors_producer_flows_folderpath)
    save_df_to_csv(df_consumer, Path('Consumer dividends received.csv'), vectors_consumer_flows_folderpath)
    
    print('\n-')

C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\matrices_dividends\Surplus of corporate producers to consumers (dividends).csv
Saved as: C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_producer_flows\Producer dividends paid.csv
Saved as: C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_consumer_flows\Consumer dividends received.csv

-


In [8]:
df_producer

,prod_muni,prod_sector,prod_cell,surplus_
0,101,s01,r101_s01,4.729443e+07
1,101,s02,r101_s02,7.684627e+07
2,101,s03,r101_s03,1.788792e+08
3,101,s04,r101_s04,3.584703e+08
4,101,s05,r101_s05,5.034090e+07
...,...,...,...,...
2641,860,s26,r860_s26,4.068831e+06
2642,860,s27,r860_s27,1.011871e+07
2643,860,s77,r860_s77,0.000000e+00
2644,860,s88,r860_s88,0.000000e+00


In [9]:
df_consumer

,cust_muni,cust_sector,cust_cell,surplus_
0,101,s01,r101_s01,3.197185e+07
1,101,s02,r101_s02,2.995070e+07
2,101,s03,r101_s03,1.377367e+07
3,101,s04,r101_s04,2.267070e+08
4,101,s05,r101_s05,1.001325e+07
...,...,...,...,...
2739,860,s24,r860_s24,8.390756e+06
2740,860,s25,r860_s25,3.411165e+05
2741,860,s26,r860_s26,3.606412e+06
2742,860,s27,r860_s27,2.634117e+06


# Process vectors

## Process individual vectors (foreign consumer spending)

In [10]:
for vector_individual_filepath in vectors_individual_folderpath.iterdir():
    print(vector_individual_filepath)
    df = pd.read_csv(vector_individual_filepath, low_memory=False)
    
    # Create a new df where munis are converted to names, then save as csv
    df_merge = convert_munis_to_names(df, muni_names_crosswalk)
    save_df_to_csv(df_merge, vector_individual_filepath, output_folderpath)
    
    print('\n-')

C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_individual\Foreign consumer spending (predicted).csv
Initial state:
(2744, 4) Index(['cust_muni', 'cust_sector', 'cust_cell', 's_foreign_'], dtype='object')
After removing 'cell' variables:
(2744, 3) Index(['cust_muni', 'cust_sector', 's_foreign_'], dtype='object')
cust_muni       string
cust_sector     object
s_foreign_     float64
dtype: object
Processing: cust_muni
Before merge: (2744, 3) Index(['cust_muni', 'cust_sector', 's_foreign_'], dtype='object')
After merge: (2744, 3) Index(['cust_muni', 'cust_sector', 's_foreign_', 'code', 'name'], dtype='object')
After processing: (2744, 3) Index(['cust_sector', 's_foreign_', 'cust_muni_name'], dtype='object')
Saved as: C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\output_csv\Foreign consumer spending (predicted).csv

-


## Process consumer flows vectors
Note: surplus of owner-occupied housing to consumers has both cust and prod cell in the raw data, but the prod cell is always cust muni + s88, so we will remove it. This effectively reduces the matrix into a vector, which will be added to the list of consumer flow vectors.

In [11]:
def get_var_name_in_table(df):
    for col in df.columns:
        if not('_muni' in col) and not('_sector' in col):
            return col

In [12]:
def process_vector(vector_filepath, player):
    # The csv files come in different types. Process each accordingly.
    # The goal is to get 3 columns per df: cust_muni, cust_sector, and the variable of interest.
    print(vector_filepath)
    df = pd.read_csv(vector_filepath, low_memory=False, header=None)
    var_name = vector_filepath.stem

    if df.shape[0] < df.shape[1]:
        df = df.transpose()

    if df.shape[1] > 2:
        df = pd.read_csv(vector_filepath, low_memory=False)
        # To deal with surplus of owner-occupied housing
        if 'cust_cell' in df.columns:
            df = df.drop('cust_cell', axis=1)
        if 'prod_cell' in df.columns:
            df = df.drop('prod_cell', axis=1) 
        var_name_in_table = get_var_name_in_table(df)
        # Process muni and sector
        df[f'{player}_muni'] = df[f'{player}_muni'].astype('str')
        df[f'{player}_sector'] = df[f'{player}_sector'].str.replace('s','').astype('str')
        df['var_name'] = var_name
        print(f"Variable names: {var_name} (csv file stem), {var_name_in_table} (col header)")
        df = df.rename(columns={var_name_in_table:'value'})
    elif df.shape[1] == 2:
        df.columns = [f'{player}_cell', 'value'] 
        df['var_name'] = var_name
        df[f'{player}_muni'] = df[f'{player}_cell'].str[1:4].astype('str')
        df[f'{player}_sector'] = df[f'{player}_cell'].str[-2:].astype('str')
        df = df.drop(f'{player}_cell', axis=1)
    
    print("Processed df_vector.shape:", df.shape)
    return df

In [13]:
def compile_vectors_into_df(vectors_consumer_flows_folderpath, player):
    df = pd.DataFrame()
    for vector_filepath in vectors_consumer_flows_folderpath.iterdir():
        if vector_filepath.suffix == '.csv':
            df_vector = process_vector(vector_filepath, player)
            if df.shape == (0,0):
                df = df_vector
                print("df.shape:", df.shape)
            else:
                df = pd.concat([df, df_vector], axis=0)
                print("df.shape:", df.shape)
    return df

In [14]:
df = compile_vectors_into_df(vectors_consumer_flows_folderpath, player='cust')
df_merge = convert_munis_to_names(df, muni_names_crosswalk)
save_df_to_csv(df_merge, Path('Additional consumer flows.csv'), output_folderpath)
df_merge

C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_consumer_flows\Consumer adjustment for pension entitlements received.csv
Processed df_vector.shape: (2744, 4)
df.shape: (2744, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_consumer_flows\Consumer dividends received.csv
Variable names: Consumer dividends received (csv file stem), surplus_ (col header)
Processed df_vector.shape: (2744, 4)
df.shape: (5488, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_consumer_flows\Consumer gross saving.csv
Processed df_vector.shape: (2744, 4)
df.shape: (8232, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_consumer_flows\Consumer interest paid.csv
Processed df_vector.shape: (2744, 4

,value,var_name,cust_sector,cust_muni_name
0,197366365.711175,Consumer adjustment for pension entitlements r...,01,København
1,308340477.702592,Consumer adjustment for pension entitlements r...,02,København
2,199690524.2474,Consumer adjustment for pension entitlements r...,03,København
3,1347486676.418711,Consumer adjustment for pension entitlements r...,04,København
4,223900554.813007,Consumer adjustment for pension entitlements r...,05,København
...,...,...,...,...
41155,90112792.0,Surplus of owner-occupied housing to consumers,24,Hjørring
41156,1237044.5,Surplus of owner-occupied housing to consumers,25,Hjørring
41157,49198124.0,Surplus of owner-occupied housing to consumers,26,Hjørring
41158,32496032.0,Surplus of owner-occupied housing to consumers,27,Hjørring


## Process producer flows vectors

In [15]:
df = compile_vectors_into_df(vectors_producer_flows_folderpath, player='prod')
df_merge = convert_munis_to_names(df, muni_names_crosswalk)
save_df_to_csv(df_merge, Path('Additional producer flows.csv'), output_folderpath)
df_merge

C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_producer_flows\Producer dividends paid.csv
Variable names: Producer dividends paid (csv file stem), surplus_ (col header)
Processed df_vector.shape: (2646, 4)
df.shape: (2646, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_producer_flows\Producer exports.csv
Processed df_vector.shape: (2646, 4)
df.shape: (5292, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_producer_flows\Producer imports.csv
Processed df_vector.shape: (2646, 4)
df.shape: (7938, 4)
C:\Users\jasonjia\Dropbox\shared_spaces\Jason-Kilian\Danish Macro\disaggregated_accounts\process_data_for_tableau\data\vectors_producer_flows\Producer product taxes paid.csv
Processed df_vector.shape: (2646, 4)
df.shape: (10584, 4)
C:\Users\jasonjia\Drop

,prod_sector,value,var_name,prod_muni_name
0,01,47294433.318169,Producer dividends paid,København
1,02,76846268.746891,Producer dividends paid,København
2,03,178879153.746721,Producer dividends paid,København
3,04,358470271.751159,Producer dividends paid,København
4,05,50340900.029888,Producer dividends paid,København
...,...,...,...,...
15871,26,1267504128.0,Producer sales to government,Hjørring
15872,27,1637282176.0,Producer sales to government,Hjørring
15873,77,0.0,Producer sales to government,Hjørring
15874,88,0.0,Producer sales to government,Hjørring
